In [2]:
!python3 --version

Python 3.10.12


In [ ]:
!brew install pyenv
!pyenv install 3.10.12
!$HOME/.pyenv/versions/3.10.12/python3 -m venv ./myenv
!source ./myenv/bin/activate
!pip install --upgrade pip
!pip install --upgrade jupyterlab
!pip install --upgrade jupyterlab-git

In [4]:
!pip3 install -U numpy matplotlib graphviz torch --quiet

In [2]:
class Value:

    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Value(data={self.data})"

In [3]:
a = Value(2)
a

Value(data=2)